In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('all_bikez_curated.csv')
df2 = pd.read_csv('all_bikez_raw.csv')

C:\Users\Jelle\AppData\Local\Temp\ipykernel_22568\393799149.py:4: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('all_bikez_curated.csv')
C:\Users\Jelle\AppData\Local\Temp\ipykernel_22568\393799149.py:5: DtypeWarning: Columns (80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('all_bikez_raw.csv')


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38472 entries, 0 to 38471
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Brand                38472 non-null  object 
 1   Model                38444 non-null  object 
 2   Year                 38472 non-null  int64  
 3   Category             38472 non-null  object 
 4   Rating               21788 non-null  float64
 5   Displacement (ccm)   37461 non-null  float64
 6   Power (hp)           26110 non-null  float64
 7   Torque (Nm)          16634 non-null  float64
 8   Engine cylinder      38461 non-null  object 
 9   Engine stroke        38461 non-null  object 
 10  Gearbox              32675 non-null  object 
 11  Bore (mm)            28689 non-null  float64
 12  Stroke (mm)          28689 non-null  object 
 13  Fuel capacity (lts)  31704 non-null  float64
 14  Fuel system          27844 non-null  object 
 15  Fuel control         22008 non-null 

In [3]:
df2 = df2.rename(columns={'Model': 'Brand_Model'})
df['Brand_Model'] = (df['Brand'] + " " + df['Model']).str.lower()

# Select the columns you want to keep from df2 and rename them
df2['Brand_Model'] = df2['Brand_Model'].str.lower()

# Select the columns you want to keep from df2 and rename them
df2_selected = df2[['Brand_Model', 'Top speed', '0-100 km/h (0-62 mph)', '1/4 mile (0.4 km)', 'Fuel consumption', 'Emission details']].rename(columns={'Top Speed': 'Speed', '0-100 km/h (0-62 mph)': 'Hundred', '1/4 mile (0.4 km)': 'Quarter'})

# Merge df1 and df2_selected on the 'Brand_Model' column
df = pd.merge(df, df2_selected, on='Brand_Model')

df['Top speed'] = df['Top speed'].str.extract('(\d+\.\d+)').astype(float)
df['Hundred'] = df['Hundred'].str.extract('(\d+\.\d+)').astype(float)
df['Quarter'] = df['Quarter'].str.extract('(\d+\.\d+)').astype(float)
df['Fuel consumption'] = df['Fuel consumption'].str.extract('(\d+\.\d+)').astype(float)

In [4]:
df['Emission details'] = df['Emission details'].astype(str)

# Define a function to categorize the emissions details
def categorize_emissions(details):
    for i in range(1, 6):  # Adjust the range based on the Euro standards you're interested in
        if details != 'nan':
            if f'Euro {i}' in details:
                return f'Euro {i}'
    return ''

# Apply the function to the 'Emissions details' column
df['Emission details'] = df['Emission details'].apply(categorize_emissions)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163792 entries, 0 to 163791
Data columns (total 34 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Brand                163792 non-null  object 
 1   Model                163792 non-null  object 
 2   Year                 163792 non-null  int64  
 3   Category             163792 non-null  object 
 4   Rating               100604 non-null  float64
 5   Displacement (ccm)   162075 non-null  float64
 6   Power (hp)           105945 non-null  float64
 7   Torque (Nm)          63436 non-null   float64
 8   Engine cylinder      163759 non-null  object 
 9   Engine stroke        163759 non-null  object 
 10  Gearbox              140454 non-null  object 
 11  Bore (mm)            133474 non-null  float64
 12  Stroke (mm)          133474 non-null  object 
 13  Fuel capacity (lts)  137422 non-null  float64
 14  Fuel system          120476 non-null  object 
 15  Fuel control     

In [6]:
df = df.dropna(subset=['Displacement (ccm)', 'Category', 'Dry weight (kg)'])
df = df[df['Displacement (ccm)'] >= 125]
df = df[df['Year'] >= 1980]
df = df[df['Dry weight (kg)'] >= 130]
df = df[df['Dry weight (kg)'] <= 418]
df = df[df['Power (hp)'] <= 325]
df = df[df['Power (hp)'] >= 14]

In [7]:
categories = df['Category'].unique()

# Print the categories
print(categories)

['ATV' 'Scooter' 'Prototype / concept model' 'Enduro / offroad'
 'Custom / cruiser' 'Naked bike' 'Touring' 'Sport touring' 'Sport'
 'Super motard' 'Allround' 'Unspecified category' 'Classic'
 'Cross / motocross' 'Trial']


In [8]:
df['Category'] = df['Category'].replace(['Sport touring'], 'Touring')
df['Category'] = df['Category'].replace(['Allround', 'Super motard', 'Enduro / offroad'], 'Dual purpose')
df['Category'] = df['Category'].replace(['Classic', 'Custom / cruiser'], 'Cruiser')
df['Category'] = df['Category'].replace(['Naked bike'], 'Standard')

df = df[~df['Category'].isin(['Prototype / concept model', 'Scooter', 'ATV', 'Cross / motocross', 'Minibike, sport', 'Minibike, cross', 'Trial', 'Speedway', 'Unspecified category'])]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27847 entries, 1178 to 162966
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Brand                27847 non-null  object 
 1   Model                27847 non-null  object 
 2   Year                 27847 non-null  int64  
 3   Category             27847 non-null  object 
 4   Rating               21696 non-null  float64
 5   Displacement (ccm)   27847 non-null  float64
 6   Power (hp)           27847 non-null  float64
 7   Torque (Nm)          25197 non-null  float64
 8   Engine cylinder      27831 non-null  object 
 9   Engine stroke        27831 non-null  object 
 10  Gearbox              26180 non-null  object 
 11  Bore (mm)            20108 non-null  float64
 12  Stroke (mm)          20108 non-null  object 
 13  Fuel capacity (lts)  22172 non-null  float64
 14  Fuel system          20793 non-null  object 
 15  Fuel control         16420 non-null  

In [10]:
df.head()

,Brand,Model,Year,Category,Rating,Displacement (ccm),Power (hp),Torque (Nm),Engine cylinder,Engine stroke,...,Rear tire,Front suspension,Rear suspension,Color options,Brand_Model,Top speed,Hundred,Quarter,Fuel consumption,Emission details
1178,ajp,pr7 adventure 650,2018,Dual purpose,3.7,659.7,48.0,58.0,Single cylinder,four-stroke,...,140/80-18,ZF Sachs Ø48mm fully adjustable,ZF Sachs progressive system with reservatory f...,White/red/black,ajp pr7 adventure 650,NaN,NaN,NaN,NaN,
1179,ajp,pr7 adventure 650,2018,Dual purpose,3.7,659.7,48.0,58.0,Single cylinder,four-stroke,...,140/80-18,ZF Sachs Ø48mm fully adjustable,ZF Sachs progressive system with reservatory f...,White/red/black,ajp pr7 adventure 650,NaN,NaN,NaN,NaN,
1180,ajp,pr7 adventure 650,2018,Dual purpose,3.7,659.7,48.0,58.0,Single cylinder,four-stroke,...,140/80-18,ZF Sachs Ø48mm fully adjustable,ZF Sachs progressive system with reservatory f...,White/red/black,ajp pr7 adventure 650,NaN,NaN,NaN,NaN,
1181,ajp,pr7 adventure 650,2019,Dual purpose,3.4,659.7,48.0,58.0,Single cylinder,four-stroke,...,140/80-18,ZF Sachs Ø48mm fully adjustable,ZF Sachs progressive system with reservatory f...,White/red/black,ajp pr7 adventure 650,NaN,NaN,NaN,NaN,
1182,ajp,pr7 adventure 650,2019,Dual purpose,3.4,659.7,48.0,58.0,Single cylinder,four-stroke,...,140/80-18,ZF Sachs Ø48mm fully adjustable,ZF Sachs progressive system with reservatory f...,White/red/black,ajp pr7 adventure 650,NaN,NaN,NaN,NaN,


In [11]:
count = df.groupby(['Category']).size()

print(count)

Category
Cruiser         6692
Dual purpose    5371
Sport           5538
Standard        6325
Touring         3921
dtype: int64


In [12]:
count = df.groupby(['Emission details']).size()

print(count)

Emission details
          25300
Euro 2        7
Euro 3      963
Euro 4     1211
Euro 5      366
dtype: int64


In [13]:
# Calculate the overall min and max weights
min_weight = df['Dry weight (kg)'].min()
max_weight = df['Dry weight (kg)'].max()

# Calculate the thresholds for 'low', 'medium', and 'high'
low_threshold = min_weight + (max_weight - min_weight) / 3
high_threshold = min_weight + 2 * (max_weight - min_weight) / 3

# Create a new column 'weight_category' based on the thresholds
df['Weight Category'] = pd.cut(df['Dry weight (kg)'], bins=[np.NINF, low_threshold, high_threshold, np.PINF], labels=['Light', 'Medium', 'Heavy'])

# Count the number of each category in 'weight_category'
count = df.groupby(['Category', 'Weight Category']).size()

print(count)

Category      Weight Category
Cruiser       Light              3088
              Medium             3012
              Heavy               592
Dual purpose  Light              4920
              Medium              446
              Heavy                 5
Sport         Light              5189
              Medium              338
              Heavy                11
Standard      Light              5842
              Medium              471
              Heavy                12
Touring       Light              1327
              Medium             1267
              Heavy              1327
dtype: int64


C:\Users\Jelle\AppData\Local\Temp\ipykernel_22568\3680721710.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count = df.groupby(['Category', 'Weight Category']).size()


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27847 entries, 1178 to 162966
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Brand                27847 non-null  object  
 1   Model                27847 non-null  object  
 2   Year                 27847 non-null  int64   
 3   Category             27847 non-null  object  
 4   Rating               21696 non-null  float64 
 5   Displacement (ccm)   27847 non-null  float64 
 6   Power (hp)           27847 non-null  float64 
 7   Torque (Nm)          25197 non-null  float64 
 8   Engine cylinder      27831 non-null  object  
 9   Engine stroke        27831 non-null  object  
 10  Gearbox              26180 non-null  object  
 11  Bore (mm)            20108 non-null  float64 
 12  Stroke (mm)          20108 non-null  object  
 13  Fuel capacity (lts)  22172 non-null  float64 
 14  Fuel system          20793 non-null  object  
 15  Fuel control        

In [15]:
df.sort_values('Fuel consumption', ascending=True)[['Brand', 'Model', 'Displacement (ccm)', 'Power (hp)', 'Top speed', 'Hundred', 'Quarter', 'Emission details']].head(20)

,Brand,Model,Displacement (ccm),Power (hp),Top speed,Hundred,Quarter,Emission details
48443,honda,cb unicorn dazzler,149.1,14.0,NaN,NaN,NaN,
48410,honda,cb hornet 160r,162.7,15.6,NaN,NaN,NaN,
48409,honda,cb hornet 160r,162.7,15.6,NaN,NaN,NaN,
48415,honda,cb hornet 160r,162.7,14.9,NaN,NaN,NaN,
48414,honda,cb hornet 160r,162.7,14.9,NaN,NaN,NaN,
48413,honda,cb hornet 160r,162.7,14.9,NaN,NaN,NaN,
48411,honda,cb hornet 160r,162.7,15.6,NaN,NaN,NaN,
48412,honda,cb hornet 160r,162.7,15.6,NaN,NaN,NaN,
48407,honda,cb hornet 160r,162.7,15.6,NaN,NaN,NaN,
48408,honda,cb hornet 160r,162.7,15.6,NaN,NaN,NaN,


In [16]:
max_weight

417.0

In [17]:
# Create a new column 'Engine Size Category' based on the engine sizes
df['Engine Size Category'] = pd.cut(df['Displacement (ccm)'], bins=[np.NINF, 400, 800, np.PINF], labels=['Small', 'Medium', 'Large'])

# Group the data by 'Motorcycle Type' and 'Engine Size Category' and count the number of each group
count = df.groupby(['Category', 'Engine Size Category']).size()

print(count)

Category      Engine Size Category
Cruiser       Small                    882
              Medium                  1729
              Large                   4081
Dual purpose  Small                    488
              Medium                  2927
              Large                   1956
Sport         Small                    813
              Medium                  1558
              Large                   3167
Standard      Small                    421
              Medium                  2269
              Large                   3635
Touring       Small                     46
              Medium                   709
              Large                   3166
dtype: int64


C:\Users\Jelle\AppData\Local\Temp\ipykernel_22568\1545498847.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count = df.groupby(['Category', 'Engine Size Category']).size()


In [18]:
# Calculate the overall min and max power
min_power = df['Power (hp)'].min()
max_power = df['Power (hp)'].max()

# Calculate the thresholds for 'low', 'medium', and 'high'
low_threshold = min_power + (max_power - min_power) / 3
high_threshold = min_power + 2 * (max_power - min_power) / 3

# Create a new column 'Power Category' based on the power
df['Power Category'] = pd.cut(df['Power (hp)'], bins=[np.NINF, low_threshold, high_threshold, np.PINF], labels=['Low', 'Medium', 'High'])

# Group the data by 'Category', 'Displacement', and 'Power Category' and count the number of each group
count = df.groupby(['Engine Size Category', 'Category', 'Power Category']).size()

print(count)

Engine Size Category  Category      Power Category
Small                 Cruiser       Low                882
                                    Medium               0
                                    High                 0
                      Dual purpose  Low                488
                                    Medium               0
                                    High                 0
                      Sport         Low                813
                                    Medium               0
                                    High                 0
                      Standard      Low                421
                                    Medium               0
                                    High                 0
                      Touring       Low                 46
                                    Medium               0
                                    High                 0
Medium                Cruiser       Low               1706
     

C:\Users\Jelle\AppData\Local\Temp\ipykernel_22568\2503489487.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count = df.groupby(['Engine Size Category', 'Category', 'Power Category']).size()


In [19]:
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame, 'Weight Category' and 'Power Category' are the columns with categories, and '0-100' is the column with 0-100 times

# Calculate the overall min and max 0-100 times
min_time = df['Top speed'].min()
max_time = df['Top speed'].max()

# Calculate the thresholds for 'fast', 'medium', and 'slow'
slow_threshold = 100
fast_threshold = 200

# Create a new column 'Time Category' based on the 0-100 times
df['Top speed Category'] = pd.cut(df['Top speed'], bins=[np.NINF, slow_threshold, fast_threshold, np.PINF], labels=['Slow', 'Medium', 'Fast'])

# Group the data by 'Weight Category', 'Power Category', and 'Time Category' and count the number of each group
count = df.groupby(['Weight Category', 'Power Category', 'Top speed Category']).size()

print(count)


Weight Category  Power Category  Top speed Category
Light            Low             Slow                    46
                                 Medium                3003
                                 Fast                   557
                 Medium          Slow                     0
                                 Medium                 206
                                 Fast                  2060
                 High            Slow                     0
                                 Medium                  44
                                 Fast                   214
Medium           Low             Slow                    41
                                 Medium                 751
                                 Fast                    70
                 Medium          Slow                     0
                                 Medium                 120
                                 Fast                   437
                 High            Slow           

C:\Users\Jelle\AppData\Local\Temp\ipykernel_22568\216802904.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count = df.groupby(['Weight Category', 'Power Category', 'Top speed Category']).size()


In [20]:
count = df.groupby(['Engine Size Category', 'Power Category', 'Emission details']).size()

print(count)

Engine Size Category  Power Category  Emission details
Small                 Low                                 2403
                                      Euro 2                 7
                                      Euro 3                92
                                      Euro 4               139
                                      Euro 5                 9
                      Medium                                 0
                                      Euro 2                 0
                                      Euro 3                 0
                                      Euro 4                 0
                                      Euro 5                 0
                      High                                   0
                                      Euro 2                 0
                                      Euro 3                 0
                                      Euro 4                 0
                                      Euro 5                 0


C:\Users\Jelle\AppData\Local\Temp\ipykernel_22568\1516029386.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  count = df.groupby(['Engine Size Category', 'Power Category', 'Emission details']).size()


In [21]:
fast_threshold

200

In [22]:
slow_threshold

100